In [1]:
import os
from datetime import date , datetime
from pyspark.sql.types import *
from pyspark.context import SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import *
import pyspark

## ML specific
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.classification import GBTClassifier, RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
day = '20150217'
num_of_domain = 100
region = "us"
seed = 1014
save_parquet = False

In [4]:
file_path = "s3n://log.sharethis.com/amankesarwani/" + region + "/" + str(day) + "/part-000000000001*"
stock_return_path = "s3n://log.sharethis.com/Stock_Proceesed_Minute_60_Second_Return_lag.csv"

In [5]:
"""
let's clean the stock data
"""
stock_return_raw = spark.read.csv(stock_return_path, sep=",", header=True)

## clean the time
stock_return_cleaned = \
    stock_return_raw.withColumn("TimeStamp", from_utc_timestamp(stock_return_raw.Lagged_Time, "UTC")) \
    .drop("Lagged_Time").withColumnRenamed("variable", "Ticker") \
    .withColumn("Return", col("Return").cast("Double"))

stock_return_cleaned.cache()

# we might need to broadcast this later
SP500_tickers = \
    [x.Ticker for x in stock_return_cleaned.select("Ticker").distinct().collect()] 


In [6]:
# stock_return_cleaned.select("TimeStamp").rdd.top(10)

In [7]:
"""Functions here."""

## personalized UDF: AAPL.x -> AAPL
def ricToTicker_(ric):
    try:
        return ric.split('.')[0]
    except:
        pass

ricToTicker = udf(ricToTicker_)


def explodeAndDropCol(pySparkDataFrame, col_names, alias = "tempCol"):
    """Explode a column and drop the un-exploded one."""
    return pySparkDataFrame.select('*', explode(col(col_names)) \
        .alias(alias)).drop(col_names)


def selectCompanyInfo(pySparkDataFrame, col_name = "tempCol"):
    """Further filter the nested column.""" 
    _temp = (
             pySparkDataFrame.withColumn("company_count", col(col_name).getItem("count").cast("Integer"))
             .withColumn("company_sentiment_score", col(col_name).getItem("sentiment_score").cast("Double"))
             .withColumn("Ticker", ricToTicker(col(col_name).getItem("ric")))
             .drop(col_name)
             .filter(col('Ticker').isNotNull())
             .filter(col('company_sentiment_score').isNotNull())
            )
    return _temp


def getTopDomain(refDomain_categories, num_of_domain):
    """Get top domain form the list."""
    refDomain_categories_filter = []
    for x in refDomain_categories:
        if len(refDomain_categories_filter) < num_of_domain:
            try:
                temp = (x[0]).split('.')[-2]
                if temp not in refDomain_categories_filter:
                    refDomain_categories_filter.append(temp)
            except:
                pass
    return refDomain_categories_filter


def parse_sharethis_time_(time):
    """set second = 0"""
    return datetime.strptime(time, "%Y-%m-%dT%H:%M:%S.%fZ").replace(second=0) 


parse_sharethis_time = udf(parse_sharethis_time_, TimestampType())

getHours = udf(lambda x: x.hour ,IntegerType())

In [8]:
"""
Then we process the sentiment data
"""
# load data
sharethis_json = spark.read.json(file_path)  # spark 2.0

In [9]:
# data process
temp_json_raw_1 = explodeAndDropCol(sharethis_json, "companies")  # explode the companies to multiples cols
temp_json_raw_2 = selectCompanyInfo(temp_json_raw_1)  # unnest the cols and clean the tickers
sharethis_json_cleaned = temp_json_raw_2.drop('stid').drop('url').drop('userAgent')  # drop unnecessary cols

# cache for further usage
sharethis_json_cleaned.cache()

DataFrame[browserFamily: string, channel: string, deviceType: string, mappedEvent: string, os: string, refDomain: string, shortIp: string, standardTimestamp: string, company_count: int, company_sentiment_score: double, Ticker: string]

In [10]:
## we only need SP500 tickers
sharethis_json_cleaned = sharethis_json_cleaned.filter(col("Ticker").isin(SP500_tickers))

In [11]:
"""Add dummies"""

device_categories = [u'Personal computer', u'Tablet', u'Smartphone']
mappedEvent_categories = [u'pview', u'click', u'search', u'share']

refDomain_categories = (sharethis_json_cleaned.groupBy("refDomain")
                        .count().orderBy(desc("count")).select("refDomain")
                        .collect())

refDomain_categories_filter = getTopDomain(refDomain_categories, num_of_domain)

exprs_device = [when(col("deviceType") == category, 1).otherwise(0).alias("is_device_"+category.replace (" ", "_"))
        for category in device_categories]
exprs_domain = [when(col("refDomain") == category, 1).otherwise(0).alias("is_domain_"+category)
        for category in refDomain_categories_filter]
exprs_mappedEvent = [when(col("mappedEvent") == category, 1).otherwise(0).alias("is_event_"+category)
        for category in mappedEvent_categories]

In [12]:
labeled_sharethis_json_cleaned = sharethis_json_cleaned.select("*", *exprs_device) \
        .select("*", *exprs_domain).select("*", *exprs_mappedEvent) \
        .drop("deviceType").drop("refDomain").drop("mappedEvent").drop("os").drop("browserFamily")
    
labeled_sharethis_json_final = (labeled_sharethis_json_cleaned
                      .withColumn("TimeStamp", parse_sharethis_time(col("standardTimestamp")))
                      .drop("standardTimestamp")
                     )

In [13]:
# labeled_sharethis_json_final.select("TimeStamp").rdd.top(1)

In [13]:
## join and filter
joined_dataframe = labeled_sharethis_json_final.join(stock_return_cleaned, ["TimeStamp", "Ticker"], how = "left_outer")
 
joined_dataframe_filtered = joined_dataframe.filter(col('Return').isNotNull())


In [14]:
#First we creat T/F labels for return 
joined_dataframe_filtered = \
    joined_dataframe_filtered.withColumn("Label",(joined_dataframe_filtered["Return"]>0).cast("Double"))

if save_parquet:
    joined_dataframe_filtered.write.parquet("sentiment_processed.parquet")
else:
    joined_dataframe_filtered.cache() 


NameError: name 'x_cols' is not defined

## RF part

In [22]:
"""
if save_pqrqu3:
    joined_dataframe_filtered = spark.read.parquet("sentiment_processed.parquet")
"""
x_cols

['company_count',
 'company_sentiment_score',
 'is_device_Personal_computer',
 'is_device_Tablet',
 'is_device_Smartphone',
 'is_domain_google',
 'is_domain_cnn',
 'is_domain_facebook',
 'is_domain_adspserving',
 'is_domain_frugalsource',
 'is_domain_frontdoor',
 'is_domain_mobilelikez',
 'is_domain_bing',
 'is_domain_pinterest',
 'is_domain_cookmates',
 'is_domain_adskpak',
 'is_domain_fox',
 'is_domain_yahoo',
 'is_domain_t',
 'is_domain_legacy',
 'is_domain_therisinghollywood',
 'is_domain_wcpo',
 'is_domain_stumbleupon',
 'is_domain_outbrain',
 'is_domain_beenverified',
 'is_domain_berry',
 'is_domain_sherwin-williams',
 'is_domain_taboola',
 'is_domain_tmz',
 'is_domain_freestufffinder',
 'is_domain_cymax',
 'is_domain_celebdirtylaundry',
 'is_domain_komando',
 'is_domain_co',
 'is_domain_usatoday',
 'is_domain_adp',
 'is_domain_wsbtv',
 'is_domain_wtop',
 'is_domain_bankofamerica',
 'is_domain_lovepanky',
 'is_domain_laptopmag',
 'is_domain_therealdeal',
 'is_domain_southernsaver

In [21]:
#let's look at what cols we have here...
x_cols=joined_dataframe_filtered.columns
#dropping redundant columns, need a better way...
x_cols.remove("Return")
x_cols.remove("TimeStamp")
x_cols.remove("Ticker")
x_cols.remove("channel")
x_cols.remove("shortIp")
x_cols.remove("Label")

In [17]:
#vectorize features
vectorizer = VectorAssembler(inputCols = x_cols,outputCol="Features")

#split data 2-8
(split20DF, split80DF) = joined_dataframe_filtered.randomSplit([0.2,0.8],seed)
# Let's cache these datasets for performance
testSet = split20DF
trainingSet = split80DF


In [18]:
dt = RandomForestClassifier()

dt.setLabelCol("Label").setPredictionCol("Predicted_Label").setFeaturesCol("Features")

dtPipeline=Pipeline()

dtPipeline.setStages([vectorizer,dt])

Pipeline_4600ab2e060e53245090

In [19]:
#first glimps
#NOTE THAT PIPELINE/VECTORIZER FIT CURRENTLY ONLY SUPPORT NUMERIC FEATURES??????
dt_glimps1=dtPipeline.fit(trainingSet)

In [20]:
#make prediction
#train_predictions = dt_glimps1.transform(trainingSet)
test_predictions = dt_glimps1.transform(testSet)

# Select (prediction, true label) and compute test error
predicted_label_test=test_predictions.select("Predicted_Label", "Label")
evaluator = BinaryClassificationEvaluator(labelCol= "Label", 
            rawPredictionCol="Predicted_Label",metricName="areaUnderROC")
accuracy_rf = evaluator.evaluate(predicted_label_test)
print("The area under ROC is {:.2f}".format(accuracy_rf))

The area under ROC is 0.50


In [21]:
#look at feature importance
rf_model1=dt_glimps1.stages[1]
#the important features
rf_model1.featureImportances.values
[x_cols[i] for i in rf_model1.featureImportances.indices]

['company_count',
 'company_sentiment_score',
 'is_device_Personal computer',
 'is_device_Tablet',
 'is_device_Smartphone',
 'is_event_pview',
 'is_event_click',
 'is_event_search',
 'is_event_share']

## playground

In [15]:
import plotly.plotly as py
import matplotlib.pyplot as plt
%matplotlib inline
import pandas
import seaborn

In [24]:
r=joined_dataframe_filtered.select("Return").toPandas()
s=joined_dataframe_filtered.select("is_device_Personal computer").toPandas()
#cc=joined_dataframe_filtered.select("company_count").toPandas()
plt.scatter(r,s)
plt.xlabel('Return')
plt.ylabel('is_device_Personal computer')

AnalysisException: u"cannot resolve '`is_device_Personal computer`' given input columns: [is_domain_livescience, is_domain_outbrain, is_domain_ford, is_domain_gravity, is_domain_pinterest, is_domain_adp, is_domain_frugalsource, is_domain_feedly, is_domain_taboola, is_domain_atlantablackstar, is_domain_co, is_domain_cnsnews, is_event_click, is_domain_space, is_domain_therisinghollywood, is_domain_adcdnx, is_domain_thebiglead, is_domain_komando, is_domain_pcgamer, is_domain_t, is_domain_adskpak, is_domain_ellentv, is_device_Personal_computer, is_domain_classifiedads, channel, Return, is_domain_newsbusters, is_domain_stumbleupon, is_domain_usatoday, is_domain_freestufffinder, is_domain_punchng, is_domain_dccomics, is_domain_msn, is_domain_wxyz, is_domain_berry, is_event_search, is_domain_thewrap, is_domain_google, is_domain_patheos, is_domain_pedestrian, is_domain_abc15, is_device_Smartphone, is_domain_newsarama, is_domain_vulture, is_device_Tablet, is_domain_zamm, is_domain_newsnet5, is_